# 1 - ⚡ Zap for Network Utility Maximization Basics ⚡


In this notebook, we introduce how to construct and solve NUM problems in Zap.
We will manually construct a small network with links and routes, solve the NUM problem using both CVXPY and PMP (ADMM), and analyze the results.

1. Creating a NUM Problem
2. Solving NUM
3. Analyzing Results

In [34]:
import zap
import torch

import numpy as np
import cvxpy as cp

from experiments.resource_opt_solve.benchmarks.nu_opt_benchmark import NUOptBenchmarkSet
from zap.resource_opt.nu_opt_bridge import NUOptBridge
from zap.admm import ADMMSolver


## Creating a NUM Problem

NUM problems in Zap consist of a link-route matrix, which defines the underlying links and streams, link capacities, stream weights, and a list of which traffic streams have linear utilities (Zap assumes by default that all streams have logarithmic utilities).

To demonstrate how to use Zap to solve a NUM problem, we can rely on the provided synthetic data generator which creates NUM test instances. We generate a small problem with the following parameters:

- 2000 links
- 1000 streams
- 10 links in the average stream
- Capacities uniformly in the range (0.1, 1)
- 0% of links congested
- Uniform stream weights of 1
- Logarithmic utilities for all streams

In [35]:
base_seed = 42
m = 2000 # Number of links
n = 1000 # Number of streams
avg_stream_length = 10 # Average stream uses 10 links
capacity_range = (0.1, 1)
link_congest_num_frac = 0
lin_util_frac = 0

benchmark = NUOptBenchmarkSet(num_problems=1, 
        m=m, 
        n=n, 
        avg_route_length=avg_stream_length, 
        capacity_range=capacity_range, 
        link_congest_num_frac=link_congest_num_frac,
        base_seed=base_seed)

We can extract the desired problem parameters from the synthetic data benchmark generator. 

In [36]:
R, capacities, w, linear_flow_idxs = benchmark.get_data(0)
nu_opt_params = {
    "R": R,
    "capacities": capacities,
    "w": w,
    "lin_device_idxs": linear_flow_idxs,
}

We can examine the problem data. 

In [37]:
R

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 9989 stored elements and shape (2000, 1000)>

In [38]:
capacities

array([0.83701913, 0.36974125, 0.69747999, ..., 0.4600636 , 0.5921001 ,
       0.67680407], shape=(2000,))

In [39]:
all(wi == 1 for wi in w)

True

In [40]:
linear_flow_idxs is None

True

Before we construct the NUM problem to solve, we need to finally specify how we group streams. We create a separate group for each set of streams with the same number of terminals (i.e. streams that utilize the same number of links).

In [41]:
grouping_params = {"variable_grouping_strategy": "discrete_terminal_groups"}

We now create the NUM problem. `NUOptBridge` provides the interface for taking problem data and converting it into a NUM problem that we can solve. 

In [42]:
nu_opt_bridge = NUOptBridge(nu_opt_params, grouping_params)

## Solving a NUM Problem

We can interface with the `nu_opt_bridge` object to solve our NUM problem in one of two ways:

1. `cvxpy` - This method will build a model in `cvxpy` and send it to an off-the-shelf solver, such as Clarabel. You can also use commerical solvers like MOSEK. This approach is best for small to medium problems and finds highly accurate solutions.

2. `admm` - This method will use a PyTorch implementation of the alternating direction method of multipliers (ADMM). This can be run on either a CPU or GPU and scales well to larger problems. In general, this method is only capable of finding medium accuracy solutions within a reasonable amount of time.


### Solving with CVXPY

To solve with `cvxpy`, we simply call `nu_opt_bridge.solve` and (optionally) specify a solver.

In [43]:
outcome = nu_opt_bridge.solve(solver=cp.CLARABEL)

We will shortly address how to interpret the solver output.

Because we are using the benchmark set, we can also extract the benchmark problem via CVXPY and solve it directly as well.

In [44]:
for i, prob in enumerate(benchmark):
        problem = prob

problem.solve(solver=cp.CLARABEL, verbose=False)

np.float64(-3331.2294732904547)

### Solving with ADMM (Proximal Message Passing)

Solving with ADMM is a little more complicated and requires two steps:

1. Transfering device data to PyTorch
2. Initializing an ADMM solver object.

In [45]:
machine = "cpu" # Pick "cuda" for a machine with an NVIDIA GPU
dtype = torch.float32
admm_devices = [d.torchify(machine=machine, dtype=dtype) for d in nu_opt_bridge.devices]
admm = ADMMSolver(
    machine=machine,
    dtype=dtype,
    atol=1e-6,
    rtol=1e-6,
    tau=2,
    alpha=1.6,
    num_iterations=1000
)
solution_admm, history_admm = admm.solve(nu_opt_bridge.net, admm_devices, nu_opt_bridge.time_horizon)

ADMM converged in 465 iterations.


## Analyzing Results

We now show how to interpret the results of the solvers. 